In [ ]:
# Make sure you're on Python > 3.8
# !pip install -r requirements.txt --quiet

In [1]:
from collections import OrderedDict

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

import flwr as fl
from flwr.simulation import run_simulation
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents

/home/rivlanm/miniconda3/envs/idlf24/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-06 16:16:06,199	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
DEVICE = torch.device('cpu')

## Load and Pre Process Data

In [3]:
!mkdir '.kaggle'
!mkdir '.kaggle/data'

with open(".kaggle/kaggle.json", 'a+') as f:
    f.write('{"username":"rajaxarcmu","key":"68d40c5e38e1c786ab57736bc5c9b2cb"}')
    
!chmod 600 '.kaggle/kaggle.json'
!kaggle datasets download -d 'danofer/compass'
!unzip -qo compass.zip -d '.kaggle/data'

mkdir: cannot create directory ‘.kaggle’: File exists
mkdir: cannot create directory ‘.kaggle/data’: File exists
compass.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!ls .kaggle/data

compas-scores-raw.csv	cox-violent-parsed_filt.csv
cox-violent-parsed.csv	propublicaCompassRecividism_data_fairml.csv


In [5]:
df = pd.read_csv('.kaggle/data/propublicaCompassRecividism_data_fairml.csv/propublica_data_for_fairml.csv')
print(df.shape)

(6172, 12)


In [6]:
df['caucasian'] = ((df['African_American'] + df['Asian'] + df['Hispanic'] + df['Native_American'] + df['Other']) == 0).astype(int)

In [ ]:
df

In [7]:
NUM_CLIENTS = 10
# REPRESENTS SILO'D ORGANIZATIONS

In [8]:
from datasets import Dataset
from flwr_datasets.partitioner import DirichletPartitioner

In [10]:
trainset, testset = train_test_split(df, test_size=0.2)
batch_size = 32

ds = Dataset.from_pandas(trainset)

partitioner = DirichletPartitioner(
    num_partitions=NUM_CLIENTS,
    partition_by="caucasian",
    alpha=0.5,
    min_partition_size=(len(trainset) // (4 * NUM_CLIENTS)),
    self_balancing=True,
    shuffle=True
)

partitioner.dataset = ds
datasets = []
for i in range(NUM_CLIENTS):
    curr_partition = partitioner.load_partition(i)
    datasets.append(curr_partition.to_pandas())

train_loaders = []
val_loaders = []

feature_columns = ['Number_of_Priors', 'score_factor','Age_Above_FourtyFive', 'Age_Below_TwentyFive', 'Misdemeanor']

for ds in datasets:
    train_x = ds[feature_columns].values
    train_y = ds['Two_yr_Recidivism'].values
    sensitive_feature = ds['caucasian'].values

    train_x, val_x, train_y, val_y, sensitive_train, sensitive_val = train_test_split(
        train_x, train_y, sensitive_feature, test_size=0.25, shuffle=True, stratify=train_y, random_state=42
    )
    
    train_x_tensor = torch.from_numpy(train_x).float()
    train_y_tensor = torch.from_numpy(train_y).float()
    sensitive_train_tensor = torch.from_numpy(sensitive_train).float()

    valid_x_tensor = torch.from_numpy(val_x).float()
    valid_y_tensor = torch.from_numpy(val_y).float()
    sensitive_val_tensor = torch.from_numpy(sensitive_val).float()

    # Create TensorDataset and DataLoader, including the sensitive attribute
    train_dataset = TensorDataset(train_x_tensor, train_y_tensor, sensitive_train_tensor)
    valid_dataset = TensorDataset(valid_x_tensor, valid_y_tensor, sensitive_val_tensor)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(valid_dataset, batch_size=batch_size)

    train_loaders.append(train_loader)
    val_loaders.append(val_loader)

# For test data
test_x = testset[feature_columns].values
test_y = testset['Two_yr_Recidivism'].values
sensitive_test = testset['caucasian'].values

test_x_tensor = torch.from_numpy(test_x).float()
test_y_tensor = torch.from_numpy(test_y).float()
sensitive_test_tensor = torch.from_numpy(sensitive_test).float()

test_dataset = TensorDataset(test_x_tensor, test_y_tensor, sensitive_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

/home/rivlanm/miniconda3/envs/idlf24/lib/python3.8/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (123) was not satisfied for alpha ([0.5]) after 0 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a desired behavior. It is recommended to adjust the alpha or min_partition_size instead. Generating partitions by sampling from a list of very wide range of alpha values can be hard to achieve. Try reducing the range between maximum (0.5) and minimum alpha (0.5) values or increasing all the values.
  warnings.warn(
/home/rivlanm/miniconda3/envs/idlf24/lib/python3.8/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (123) was not satisfied for alpha ([0.5]) after 1 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet dist

## Client Model Architecture

In [ ]:
class BaselineNN(nn.Module):
    def __init__(self):
        super(BaselineNN, self).__init__()
        self.fc1 = nn.Linear(5, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

def compute_eod(preds, labels, sensitive_feature):
    preds_binary = (preds >= 0.5).float()
    y_true_mask = (labels == 1).view(-1)

    p_a0 = preds_binary[y_true_mask & (sensitive_feature == 0)].mean().item()
    p_a1 = preds_binary[y_true_mask & (sensitive_feature == 1)].mean().item()

    eod = p_a0 - p_a1
    return eod

def train(net, trainloader, epochs, verbose=True):
    """
    Train Network on Training Set
    """
    criterion = nn.BCELoss()
    optimizer = optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        all_preds, all_labels, all_sensitives = [], [], []
        
        for inputs, labels, sensitive_features in trainloader:
            inputs, labels, sensitive_features = inputs.to(DEVICE), labels.to(DEVICE), sensitive_features.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(inputs)
            labels = labels.view(-1, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * inputs.size(0)
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Append predictions and sensitive data for EOD computation
            all_preds.append(outputs.detach().cpu())
            all_labels.append(labels.detach().cpu())
            all_sensitives.append(sensitive_features.cpu())
        
        # Compute EOD at the end of the epoch
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)
        all_sensitives = torch.cat(all_sensitives)
        
        eod = compute_eod(all_preds, all_labels, all_sensitives)
        
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f} - Acc: {epoch_acc:.4f} - EOD: {eod:.4f}")

def test(net, testloader, verbose=True):
    criterion = nn.BCELoss()
    net.eval()
    correct, total, loss = 0, 0, 0.0
    all_preds, all_labels, all_sensitives = [], [], []
    
    with torch.no_grad():
        for inputs, labels, sensitive_features in testloader:
            inputs, labels, sensitive_features = inputs.to(DEVICE), labels.to(DEVICE), sensitive_features.to(DEVICE)
            outputs = net(inputs)
            labels = labels.view(-1, 1)
            loss += criterion(outputs, labels).item() * inputs.size(0)
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Append predictions and sensitive data for EOD computation
            all_preds.append(outputs.detach().cpu())
            all_labels.append(labels.detach().cpu())
            all_sensitives.append(sensitive_features.cpu())
    
    # Compute EOD at the end of testing
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    all_sensitives = torch.cat(all_sensitives)
    
    eod = compute_eod(all_preds, all_labels, all_sensitives)
    
    loss /= len(testloader.dataset)
    acc = correct / total
    if verbose:
        print(f"Test Loss: {loss:.4f} - Acc: {acc:.4f} - EOD: {eod:.4f}")
    return loss, acc, eod

# Centralized Learning

In [ ]:
model = BaselineNN()

In [ ]:
for i in range(NUM_CLIENTS):
    train_loader = train_loaders[i]
    val_loader = val_loaders[i]
    model = model.to(DEVICE)
    epochs = 10

    for epoch in range(epochs):
        train(model, train_loader, 1, verbose=False)
        loss, acc, eod = test(model, val_loader, verbose=False)

    loss, acc, eod = test(model, test_loader, verbose=False)
    print(f"Client {i} - Test Loss: {loss:.4f} - Acc: {acc:.4f} - EOD: {eod:.4f}")

# Fairness Eval - Centralized

# Federated Learning with Flower

In [11]:
import flwr as fl
from flwr.simulation import run_simulation
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents

from collections import OrderedDict
import numpy as np
import pandas as pd
import torch

from custom_flwr.server_app import server_fn as server_fn_custom
from custom_flwr.client_app import client_fn as client_fn_custom

DEVICE = torch.device('cpu')

def server_fn(context: Context):
    context.run_config = {
        'num-server-rounds' : 10,
        'fraction-fit': 1,
        'fraction-evaluate': 1,
        'local-epochs': 2,
        'server-device': str(DEVICE),
        'use-wandb': False
    }
    return server_fn_custom(context)

def client_fn(context: Context):
    return client_fn_custom(context)

client = ClientApp(client_fn=client_fn)
server = ServerApp(server_fn=server_fn)


backend_config = {"client_resources": None}
NUM_PARTITIONS = 10
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_PARTITIONS,
    backend_config=backend_config,
)

/home/rivlanm/miniconda3/envs/idlf24/lib/python3.8/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (61) was not satisfied for alpha ([0.5]) after 0 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a desired behavior. It is recommended to adjust the alpha or min_partition_size instead. Generating partitions by sampling from a list of very wide range of alpha values can be hard to achieve. Try reducing the range between maximum (0.5) and minimum alpha (0.5) values or increasing all the values.
  warnings.warn(
/home/rivlanm/miniconda3/envs/idlf24/lib/python3.8/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (61) was not satisfied for alpha ([0.5]) after 1 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distri

Test Accuracy: 0.5562753036437247 - Test Loss: 25.511822222135006 - EOD: 0.07481992244720459 - IndFair: 0.5877457112073898


INFO :      configure_fit: strategy sampled 10 clients (out of 10)
(ClientAppActor pid=259346) /home/rivlanm/miniconda3/envs/idlf24/lib/python3.8/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (61) was not satisfied for alpha ([0.5]) after 0 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a desired behavior. It is recommended to adjust the alpha or min_partition_size instead. Generating partitions by sampling from a list of very wide range of alpha values can be hard to achieve. Try reducing the range between maximum (0.5) and minimum alpha (0.5) values or increasing all the values.
(ClientAppActor pid=259346)   warnings.warn(


(ClientAppActor pid=259346) Avg Train Loss: 0.7525189995765686 - EOD: nan - Accuracy: 0.5849673202614379 - Ind Fair: 2.1438612937927246
(ClientAppActor pid=259348) Avg Train Loss: 0.8499585926532746 - EOD: 0.17518247663974762 - Accuracy: 0.5511551155115512 - Ind Fair: 1.4491927474737167
(ClientAppActor pid=259344) Skipping batch with single class.


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      fit progress: (1, 20.874681086112293, {'centralized_accuracy': 0.5417004048582996, 'eod': -0.006097560748457909, 'indf': 0.05039329081773758}, 18.389092947996687)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)


...

INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      fit progress: (10, 20.125912505846756, {'centralized_accuracy': 0.6720647773279352, 'eod': 0.20494157075881958, 'indf': 0.40790942311286926}, 54.64603922099923)
INFO :      configure_evaluate: strategy sampled 10 clients (out of 10)


LOGG: RESULTS
{'indfair': 0.3939465284347534, 'acc': 0.6484848484848484, 'eod': 0.0892857015132904, 'train_loss': 0.6244756976763407, 'id': 5} 165
{'indfair': 0.6376438438892365, 'acc': 0.689594356261023, 'eod': nan, 'train_loss': 0.6345612903436025, 'id': 0} 567
{'indfair': 0.44157856702804565, 'acc': 0.6990740740740741, 'eod': 0.18333332985639572, 'train_loss': 0.6119434152330671, 'id': 6} 216
{'indfair': 0.6968524381518364, 'acc': 0.6794871794871795, 'eod': 0.4285714328289032, 'train_loss': 0.5994156002998352, 'id': 1} 234
{'indfair': 0.5711197853088379, 'acc': 0.6495652173913044, 'eod': 0.01862068474292755, 'train_loss': 0.6519792758756213, 'id': 2} 1150
{'indfair': 0.7006009519100189, 'acc': 0.6932515337423313, 'eod': 0.1432507038116455, 'train_loss': 0.6380364757317764, 'id': 8} 815
{'indfair': 1.4076769948005676, 'acc': 0.6830357142857143, 'eod': nan, 'train_loss': 0.6352867313793727, 'id': 9} 224
{'indfair': 0.8938933610916138, 'acc': 0.6554809843400448, 'eod': nan, 'train_loss

INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 10 round(s) in 56.66s
INFO :      	History (loss, distributed):
INFO :      		round 1: 19.321498603787564
INFO :      		round 2: 17.977108918770366
INFO :      		round 3: 18.290612224692094
INFO :      		round 4: 18.517615220185693
INFO :      		round 5: 16.996221114865016
INFO :      		round 6: 17.126661344569452
INFO :      		round 7: 18.422375266865906
INFO :      		round 8: 17.258604381012983
INFO :      		round 9: 17.42877509300135
INFO :      		round 10: 17.533226886084154
INFO :      	History (loss, centralized):
INFO :      		round 0: 25.511822222135006
INFO :      		round 1: 20.874681086112293
INFO :      		round 2: 20.18538178388889
INFO :      		round 3: 20.10611885633224
INFO :      		round 4: 20.09442985516328
INFO :      		round 5: 20.010937603620384
INFO :      		round 6: 20.123105781200604
INFO :      		round 7: 19.963842173417408
INFO :      		

(ClientAppActor pid=259346) Skipping batch with single class.
(ClientAppActor pid=259349) Avg Train Loss: 0.6352867313793727 - EOD: nan - Accuracy: 0.6830357142857143 - Ind Fair: 1.4076769948005676 [repeated 4x across cluster]
(ClientAppActor pid=259347) Avg Train Loss: 0.6380364757317764 - EOD: 0.1432507038116455 - Accuracy: 0.6932515337423313 - Ind Fair: 0.7006009519100189 [repeated 5x across cluster]
(ClientAppActor pid=259349) Test Accuracy: 0.7183098591549296 - Test Loss: 11.671034435431162 - EOD: nan - IndFair: 0.39515921473503113 [repeated 4x across cluster]
(ClientAppActor pid=259350) Test Accuracy: 0.632183908045977 - Test Loss: 18.11169668038686 - EOD: 0.6857142746448517 - IndFair: 0.7766162157058716 [repeated 6x across cluster]


(ClientAppActor pid=259346) /home/rivlanm/miniconda3/envs/idlf24/lib/python3.8/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (61) was not satisfied for alpha ([0.5]) after 0 attempts at sampling from the Dirichlet distribution. The probability sampling from the Dirichlet distribution will be repeated. Note: This is not a desired behavior. It is recommended to adjust the alpha or min_partition_size instead. Generating partitions by sampling from a list of very wide range of alpha values can be hard to achieve. Try reducing the range between maximum (0.5) and minimum alpha (0.5) values or increasing all the values. [repeated 111x across cluster]
(ClientAppActor pid=259346)   warnings.warn( [repeated 111x across cluster]


In [ ]:
# hi